In [76]:
from google.colab import drive
drive.mount('/content/drive')

from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/'  #change dir to your project folder

import numpy as np
X_train = np.load('gdrive/My Drive/dataset1/acc+gyr/trainX.npy')
y_train = np.load('gdrive/My Drive/dataset1/acc+gyr/trainy.npy')
X_test = np.load('gdrive/My Drive/dataset1/acc+gyr/testX.npy')
y_test = np.load('gdrive/My Drive/dataset1/acc+gyr/testy.npy')
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)


Mounted at /content/drive
Mounted at /content/gdrive
(33104, 128, 6) (33104, 118) (3740, 128, 6) (3740, 118)


In [77]:
from numpy import mean
from numpy import std
from numpy import dstack
from numpy import save, load
from pandas import read_csv
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Input ,concatenate
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import LSTM, Conv1D, concatenate,GlobalMaxPooling1D,TimeDistributed, MaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import concatenate
from tensorflow.keras import regularizers

In [78]:
from sklearn.model_selection import train_test_split
X_training, X_validation, y_training, y_validation = train_test_split(X_train, y_train, test_size=0.20, random_state=42)
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]
validation_data=(X_validation, y_validation)

In [79]:
CNN_model= Sequential()
CNN_model.add(Conv1D(filters=32, kernel_size=9, strides=2, activation='relu', input_shape=(n_timesteps,n_features)))
CNN_model.add(MaxPooling1D(pool_size=2 ,strides=2))
CNN_model.add(Dropout(0.5))
CNN_model.add(Conv1D(filters=64, kernel_size=3, strides=1, activation='relu'))
CNN_model.add(Conv1D(filters=128, kernel_size=3, strides=1, activation='relu'))
CNN_model.add(MaxPooling1D(pool_size=2 ,strides=2))
CNN_model.add(Dropout(0.5))
CNN_model.add(Conv1D(filters=128, kernel_size=1, strides=1, activation='relu'))
CNN_model.add(Flatten())
CNN_model.add(Dense(n_outputs, activation='relu'))
CNN_model.summary()

LSTM_model= Sequential()
LSTM_model.add(LSTM(512,input_shape=(n_timesteps,n_features),return_sequences=True))
LSTM_model.add(LSTM(256))
LSTM_model.add(Dropout(0.2))
LSTM_model.add(Dense(128, activation='relu'))
LSTM_model.add(Dense(n_outputs, activation='relu'))
LSTM_model.summary()

inputs = Input(shape=(n_timesteps,n_features))
mergedInput= concatenate([CNN_model(inputs),LSTM_model(inputs)])
out = Dense(n_outputs, activation='softmax')(mergedInput)
model3 = Model(inputs,out)
model3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#model3.layers[0].set_weights(LSTM_model.layers[4].get_weights())
model3.summary()



Model: "sequential_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_56 (Conv1D)          (None, 60, 32)            1760      
                                                                 
 max_pooling1d_28 (MaxPoolin  (None, 30, 32)           0         
 g1D)                                                            
                                                                 
 dropout_36 (Dropout)        (None, 30, 32)            0         
                                                                 
 conv1d_57 (Conv1D)          (None, 28, 64)            6208      
                                                                 
 conv1d_58 (Conv1D)          (None, 26, 128)           24704     
                                                                 
 max_pooling1d_29 (MaxPoolin  (None, 13, 128)          0         
 g1D)                                                

In [80]:
history = model3.fit(X_training, y_training, epochs=30, verbose=True, validation_data=(X_validation, y_validation), batch_size=128)

Epoch 1/30
207/207 [==============================] - 46s 202ms/step - loss: 2.0951 - accuracy: 0.4706 - val_loss: 0.7732 - val_accuracy: 0.7996
Epoch 2/30
207/207 [==============================] - 40s 195ms/step - loss: 0.6583 - accuracy: 0.8247 - val_loss: 0.4667 - val_accuracy: 0.8783
Epoch 3/30
207/207 [==============================] - 40s 195ms/step - loss: 0.4630 - accuracy: 0.8746 - val_loss: 0.3422 - val_accuracy: 0.9103
Epoch 4/30
207/207 [==============================] - 40s 194ms/step - loss: 0.3729 - accuracy: 0.8978 - val_loss: 0.3038 - val_accuracy: 0.9160
Epoch 5/30
207/207 [==============================] - 40s 195ms/step - loss: 0.2887 - accuracy: 0.9192 - val_loss: 0.2685 - val_accuracy: 0.9230
Epoch 6/30
207/207 [==============================] - 40s 194ms/step - loss: 0.2504 - accuracy: 0.9287 - val_loss: 0.2321 - val_accuracy: 0.9340
Epoch 7/30
207/207 [==============================] - 40s 195ms/step - loss: 0.2143 - accuracy: 0.9372 - val_loss: 0.2274 - val_ac

In [81]:
loss, accuracy = model3.evaluate(X_validation, y_validation, verbose=False)
print("Validation Accuracy:  {:.4f}".format(accuracy))
loss, accuracy = model3.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy: {:.4f}".format(accuracy))

Validation Accuracy:  0.9645
Testing Accuracy: 0.9027
